In [1]:
import cv2, numpy as np
import math
import time
import random
from matplotlib import pyplot as plt
%matplotlib inline
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import argparse
import os.path

output_notebook()

Loading BokehJS ...

In [2]:
# 초기값 설정
confThreshold = 0.5  # Confidence threshold
nmsThreshold = 0.4   # Non-maximum suppression threshold
inpWidth = 416       # Width of network's input image
inpHeight = 416      # Height of network's input image

# yolo 네트워크 불러오기
classesFile = "coco.names"
classes = None
with open(classesFile, 'rt') as f : classes = f.read().rstrip('\n').split('\n')

modelConfiguration = "yolov3.cfg"
modelWeights = "yolov3.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [3]:
# 출력 레이어 가져오기
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# bounding box 그리기
def drawPred(classId, conf, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine),
                 (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)

# NMS를 적용하여 낮은 confidence의 bounding box 제거
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

In [4]:
import numpy as np
import cv2

def color_extraction(img, num) :
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

    lower_blue = (num-10, 100, 100) # hsv 이미지에서 바이너리 이미지로 생성, num = 90 -> green, num = 0 -> red
    upper_blue = (num+10, 255, 255)
    img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue) # 범위내의 픽셀들은 흰색, 나머지 검은색

    img_result = cv2.bitwise_and(img, img, mask = img_mask) 

    return img_result

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
frame = cv2.imread("traffic lights.jpg")

# plt.imshow(frame)
# plt.show()

blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

net.setInput(blob)
outs = net.forward(getOutputsNames(net))

frameHeight = frame.shape[0]
frameWidth = frame.shape[1]

classIds = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        classId = np.argmax(scores)
        confidence = scores[classId]
        if confidence > confThreshold:
            center_x = int(detection[0] * frameWidth)
            center_y = int(detection[1] * frameHeight)
            width = int(detection[2] * frameWidth)
            height = int(detection[3] * frameHeight)
            left = int(center_x - width / 2)
            top = int(center_y - height / 2)
            classIds.append(classId)
            confidences.append(float(confidence))
            boxes.append([left, top, width, height])

indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
cnt = 0
for i in indices:
    i = i[0]
    box = boxes[i]
    left = box[0]
    top = box[1]
    width = box[2]
    height = box[3]
    classId = classIds[i]
    if classId == 9 :
        img = frame.copy()
        img = img[top:top+height, left:left+width]
        img_green = color_extraction(img, 90)
        img_red = color_extraction(img, 0)
        img_yellow = color_extraction(img, 30)
        if not np.array_equal(img_green, np.zeros_like(img)) :
            cv2.putText(frame, 'green', (left, top+height+15), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
#             cv2.imshow('img'+str(cnt), img)
        elif not np.array_equal(img_red, np.zeros_like(img)) :
            cv2.putText(frame, 'red', (left, top+height+15), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
#             cv2.imshow('img'+str(cnt), img)
        elif not np.array_equal(img_yellow, np.zeros_like(img)) :
            cv2.putText(frame, 'yellow', (left, top+height+15), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,255), 2)
#             cv2.imshow('img'+str(cnt), img)
#         cv2.imshow('img'+str(cnt), img)
        cnt += 1
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
#         print([left, top, width, height])
#         plt.imshow(img)
#         plt.show()
#     drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

cv2.imshow('out', frame)
cv2.imwrite('traffic lights results.jpg', frame)
# plt.imshow(frame)
# plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
color = [0, 255, 255] # 파란색
pixel = np.uint8([[color]]) # 한픽셀로 구성된 이미지로 변환

hsv = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV) 
# cvtColor 함수를 이용하여 hsv 색공간으로 변환

hsv = hsv[0][0] # 픽셀값을 가져옴

print("bgr: ", color)
print("hsv: ", hsv) # +_ 10

bgr:  [0, 255, 255]
hsv:  [ 30 255 255]


In [34]:
import numpy as np
import cv2

img_color = cv2.imread("traffic lights.jpg") # 이미지 파일을 컬러로 불러옴
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

lower_blue = (num-10, 100, 100) # hsv 이미지에서 바이너리 이미지로 생성 , 적당한 값 30, num = 90 -> green, num = 0 -> red
upper_blue = (num+10, 255, 255)
img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue) # 범위내의 픽셀들은 흰색, 나머지 검은색

img_result = cv2.bitwise_and(img_color, img_color, mask = img_mask) 
# 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득

cv2.imshow('img_color', img_color)
cv2.imshow('img_mask', img_mask)
cv2.imshow('img_result', img_result)

cv2.waitKey(0)
cv2.destroyAllWindows()